In [ ]:
import pandas as pd
import numpy as np
import os
import itertools
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

DATA_DIR = r'C:\Users\Sahar\Desktop\Clickbait project\Dataset\data'

def load_data(sample_size=None):
    data = {}
    id_cols = ['videoid', 'channelid', 'userid', 'uuid', 'hashedvideoid']
    
    for fname in sorted(os.listdir(DATA_DIR)):
        if fname.lower().endswith('.csv'):
            fp = os.path.join(DATA_DIR, fname)
            try:
                df = pd.read_csv(fp, low_memory=False, nrows=sample_size)
                df.columns = df.columns.str.lower().str.replace('[^a-z0-9]', '', regex=True)
                for c in df.columns:
                    if c in id_cols or c.endswith('id') or c.endswith('uuid'):
                        df[c] = df[c].astype(str).str.strip()
                data[fname] = df
                print(f"Loaded {fname:30s} ({len(df):>10,} rows, {len(df.columns):>3} cols)")
            except Exception as e:
                print(f"Error reading {fname}: {e}")
    return data

print("Loading dataset...")
dataset = load_data(sample_size=None)
print(f"\nTotal files loaded: {len(dataset)}")

## 1. Data Quality Assessment

In [ ]:
def assess_data_quality(data):
    quality_report = []
    
    for fname, df in data.items():
        missing_pct = (df.isnull().sum() / len(df) * 100).to_dict()
        duplicates = df.duplicated().sum()
        
        quality_report.append({
            'File': fname,
            'Rows': len(df),
            'Columns': len(df.columns),
            'Duplicates': duplicates,
            'Max_Missing_%': max(missing_pct.values()) if missing_pct else 0,
            'Memory_MB': df.memory_usage(deep=True).sum() / 1024**2
        })
    
    quality_df = pd.DataFrame(quality_report)
    return quality_df

print("\n=== Data Quality Report ===")
quality = assess_data_quality(dataset)
print(quality.to_string(index=False))
print(f"\nTotal Memory Usage: {quality['Memory_MB'].sum():.2f} MB")

## 2. Column Relationship Mapping

In [ ]:
def map_column_relationships(data, sample_size=1000):
    relationships = []
    id_keywords = {'id', 'uuid', 'hash', 'user', 'video', 'channel'}
    
    for f1, f2 in itertools.combinations(data.keys(), 2):
        df1, df2 = data[f1], data[f2]
        
        id_cols_f1 = [c for c in df1.columns if any(kw in c.lower() for kw in id_keywords)]
        id_cols_f2 = [c for c in df2.columns if any(kw in c.lower() for kw in id_keywords)]
        
        if not id_cols_f1 or not id_cols_f2:
            continue
        
        for c1 in id_cols_f1:
            col_data = df1[c1].dropna().unique()[:sample_size]
            if len(col_data) < 2 or len(col_data) > 100000:
                continue
            set1 = set(col_data)
            
            for c2 in id_cols_f2:
                col_data2 = df2[c2].dropna().unique()[:sample_size]
                if len(col_data2) < 2 or len(col_data2) > 100000:
                    continue
                set2 = set(col_data2)
                
                intersect = set1.intersection(set2)
                if not intersect or len(intersect) < 2:
                    continue
                
                len1, len2, len_int = len(set1), len(set2), len(intersect)
                coverage = len_int / min(len1, len2) * 100
                
                rel_type = 'Intersection'
                if len1 == len2 == len_int:
                    rel_type = 'Identity'
                elif len_int == len1 < len2:
                    rel_type = 'Subset'
                
                if coverage >= 5.0:
                    relationships.append({
                        'From_File': f1,
                        'From_Column': c1,
                        'To_File': f2,
                        'To_Column': c2,
                        'Overlap_Count': len_int,
                        'Coverage_%': round(coverage, 2),
                        'Type': rel_type
                    })
    
    return pd.DataFrame(relationships)

print("\nMapping column relationships (ID columns)...")
relationships_df = map_column_relationships(dataset)
print(f"\nFound {len(relationships_df)} relationships")
print("\nTop 15 Relationships:")
print(relationships_df.nlargest(15, 'Coverage_%')[['From_File', 'From_Column', 'To_File', 'To_Column', 'Coverage_%', 'Type']].to_string(index=False))

## 3. Temporal Feature Engineering

In [ ]:
def extract_temporal_features(data):
    temporal_features = []
    
    for fname, df in data.items():
        datetime_cols = []
        
        for col in df.columns:
            if any(keyword in col.lower() for keyword in ['time', 'date', 'timestamp', 'when']):
                try:
                    pd.to_datetime(df[col], errors='coerce')
                    datetime_cols.append(col)
                except:
                    pass
        
        if datetime_cols:
            for col in datetime_cols:
                try:
                    dt = pd.to_datetime(df[col], errors='coerce')
                    valid_dates = dt.dropna()
                    
                    if len(valid_dates) > 0:
                        temporal_features.append({
                            'File': fname,
                            'Column': col,
                            'Type': 'Datetime',
                            'Non_Null_%': (len(valid_dates) / len(df) * 100),
                            'Min_Date': valid_dates.min(),
                            'Max_Date': valid_dates.max(),
                            'Date_Range_Days': (valid_dates.max() - valid_dates.min()).days
                        })
                except:
                    pass
    
    return pd.DataFrame(temporal_features)

print("\nExtracting temporal features...")
temporal_df = extract_temporal_features(dataset)
if len(temporal_df) > 0:
    print("\nTemporal Columns Found:")
    print(temporal_df[['File', 'Column', 'Non_Null_%', 'Min_Date', 'Max_Date', 'Date_Range_Days']].to_string(index=False))
else:
    print("No temporal columns detected")

## 4. Categorical Feature Analysis

In [ ]:
def analyze_categorical_features(data):
    categorical_analysis = []
    
    for fname, df in data.items():
        for col in df.select_dtypes(include=['object']).columns:
            unique_count = df[col].nunique()
            
            if unique_count <= 1000:
                categorical_analysis.append({
                    'File': fname,
                    'Column': col,
                    'Unique_Values': unique_count,
                    'Null_%': (df[col].isnull().sum() / len(df) * 100),
                    'Top_Value': df[col].value_counts().index[0] if unique_count > 0 else None,
                    'Top_Value_Freq': df[col].value_counts().values[0] if unique_count > 0 else 0
                })
    
    return pd.DataFrame(categorical_analysis).sort_values('Unique_Values')

print("\nAnalyzing categorical features...")
categorical_df = analyze_categorical_features(dataset)
print(f"\nFound {len(categorical_df)} categorical columns (with ≤1000 unique values)")
print("\nTop 20 Categorical Features by Cardinality:")
print(categorical_df.nlargest(20, 'Unique_Values')[['File', 'Column', 'Unique_Values', 'Null_%', 'Top_Value']].to_string(index=False))

## 5. Numerical Feature Analysis

## 6. Feature Engineering Opportunities

In [ ]:
def identify_feature_engineering_opportunities(data):
    opportunities = []
    
    for fname, df in data.items():
        for col in df.columns:
            dtype = df[col].dtype
            nunique = df[col].nunique()
            null_pct = (df[col].isnull().sum() / len(df) * 100)
            
            recommendations = []
            
            if dtype == 'object' and nunique <= 50:
                recommendations.append('One-Hot Encoding')
            
            if dtype == 'object' and nunique > 50 and nunique < 1000:
                recommendations.append('Target Encoding / Frequency Encoding')
            
            if pd.api.types.is_numeric_dtype(df[col]) and df[col].min() >= 0:
                if df[col].std() > 0:
                    recommendations.append('Log Transform / Scaling')
            
            if 'id' in col.lower() or 'uuid' in col.lower():
                recommendations.append('Group Statistics / Aggregation')
            
            if null_pct > 10:
                recommendations.append('Missing Value Imputation')
            
            if recommendations:
                opportunities.append({
                    'File': fname,
                    'Column': col,
                    'Data_Type': str(dtype),
                    'Unique_Values': nunique,
                    'Missing_%': round(null_pct, 2),
                    'Recommendations': ', '.join(recommendations)
                })
    
    return pd.DataFrame(opportunities)

print("\n=== Feature Engineering Opportunities ===")
opportunities_df = identify_feature_engineering_opportunities(dataset)
print(f"\nIdentified {len(opportunities_df)} columns with engineering potential\n")
for idx, row in opportunities_df.head(30).iterrows():
    print(f"{row['File']:30s} | {row['Column']:20s} | {row['Recommendations']}")

## 7. Correlation & Interaction Analysis

In [ ]:
def compute_correlation_matrix(data):
    all_numerical = pd.DataFrame()
    
    for fname, df in list(data.items())[:5]:
        numeric_cols = df.select_dtypes(include=[np.number]).columns
        if len(numeric_cols) > 0:
            print(f"\n{fname}:")
            corr_matrix = df[numeric_cols].corr()
            
            high_corr = []
            for i in range(len(corr_matrix.columns)):
                for j in range(i+1, len(corr_matrix.columns)):
                    if abs(corr_matrix.iloc[i, j]) > 0.7:
                        high_corr.append({
                            'Var1': corr_matrix.columns[i],
                            'Var2': corr_matrix.columns[j],
                            'Correlation': round(corr_matrix.iloc[i, j], 3)
                        })
            
            if high_corr:
                corr_df = pd.DataFrame(high_corr)
                print(corr_df.to_string(index=False))
            else:
                print("No high correlations (>0.7) found")

print("\n=== Correlation Analysis (First 5 Files) ===")
compute_correlation_matrix(dataset)

## 8. Summary & Insights

In [ ]:
print("\n" + "="*80)
print("FEATURE ENGINEERING SUMMARY")
print("="*80)

print(f"\n1. DATASET OVERVIEW")
print(f"   - Total Files: {len(dataset)}")
print(f"   - Total Columns: {sum(len(df.columns) for df in dataset.values())}")
print(f"   - Total Rows: {sum(len(df) for df in dataset.values()):,}")
print(f"   - Total Memory: {sum(df.memory_usage(deep=True).sum() for df in dataset.values()) / 1024**2:.2f} MB")

print(f"\n2. DATA QUALITY")
avg_duplicates = quality['Duplicates'].mean()
avg_missing = quality['Max_Missing_%'].mean()
print(f"   - Average Duplicates per File: {avg_duplicates:.1f}")
print(f"   - Average Max Missing %: {avg_missing:.2f}%")

print(f"\n3. RELATIONSHIPS")
print(f"   - Total Column Relationships: {len(relationships_df)}")
print(f"   - Identity Relationships: {len(relationships_df[relationships_df['Type']=='Identity'])}")
print(f"   - Subset Relationships: {len(relationships_df[relationships_df['Type']=='Subset'])}")
print(f"   - Intersection Relationships: {len(relationships_df[relationships_df['Type']=='Intersection'])}")

print(f"\n4. FEATURE TYPES")
print(f"   - Categorical Columns: {len(categorical_df)}")
print(f"   - Numerical Columns: {len(numerical_df)}")
print(f"   - Temporal Columns: {len(temporal_df)}")

print(f"\n5. ENGINEERING OPPORTUNITIES")
print(f"   - Total Features for Engineering: {len(opportunities_df)}")
print(f"   - Encoding Candidates: {len(opportunities_df[opportunities_df['Recommendations'].str.contains('Encoding')])}")
print(f"   - Transformation Candidates: {len(opportunities_df[opportunities_df['Recommendations'].str.contains('Transform')])}")
print(f"   - Aggregation Candidates: {len(opportunities_df[opportunities_df['Recommendations'].str.contains('Aggregation')])}")

print("\n" + "="*80)